In [1]:
from travel_optimizer import *

# utown

In [4]:
B = 150
B_min =0
M = 10000
T_start = 480
T_end = 1380
dp_node = 'utown'

itinerary, total_enjoyment = create_model(dp_node, B, M, T_start, T_end, B_min=B_min)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 37000 rows, 18901 columns and 165105 nonzeros
Model fingerprint: 0x1c4d4cd3
Variable types: 270 continuous, 18631 integer (18631 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 16112 rows and 11254 columns
Presolve time: 0.37s
Presolved: 20888 rows, 7647 columns, 74504 nonzeros
Variable types: 203 continuous, 7444 integer (7444 binary)

Root relaxation: objective 4.384801e+01, 561 iterations, 0.02 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   43.84801    0   23          -   43.84801      -     -    0s
     

# marinabay

In [3]:
B = 800
B_min = 600
M = 10000
T_start = 480
T_end = 1320
dp_node = 'marinabay'

itinerary, total_enjoyment = create_model(dp_node, B, M, T_start, T_end, B_min=B_min)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 37270 rows, 19036 columns and 183870 nonzeros
Model fingerprint: 0xb33ef6ac
Variable types: 405 continuous, 18631 integer (18631 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 13484 rows and 9215 columns
Presolve time: 0.04s

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.11 work units)
Thread count was 1 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -
Model did not find an optimal solution.
1st Round Optimization Failed


NameError: name 'exit' is not defined

# airport

In [6]:
1260/60

21.0

In [ ]:
B = 500
B_min=0
M = 10000
T_start = 840 # 2pm
T_end = 1260 #(9pm+12)*60
dp_node = 'airport'

itinerary, total_enjoyment = create_model(dp_node, B, M, T_start, T_end, B_min=0)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-23
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 37134 rows, 18902 columns and 146948 nonzeros
Model fingerprint: 0x0960503f
Variable types: 271 continuous, 18631 integer (18631 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 14607 rows and 9804 columns
Presolve time: 0.14s
Presolved: 22527 rows, 9098 columns, 76176 nonzeros
Variable types: 232 continuous, 8866 integer (8866 binary)

Root relaxation: objective 1.595000e+02, 1446 iterations, 0.03 seconds (0.09 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/